In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.tail()

,isbn,title,author
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato
271378,0767409752,A Guided Tour of Rene Descartes' Meditations o...,Christopher Biffle


In [5]:
df_ratings.tail()

,user,isbn,rating
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0
1149779,276723,05162443314,8.0


In [6]:
# get users with more than 199 ratings
users = df_ratings.groupby('user')[['rating']].count().sort_values('rating', ascending=False).reset_index()
users = users[users['rating'] > 199]
users.tail()

,user,rating
900,99955,200
901,225595,200
902,83671,200
903,252827,200
904,26883,200


In [7]:
# get books with more than 99 ratings
isbns = df_ratings.groupby('isbn')[['rating']].count().sort_values('rating', ascending=False).reset_index()
isbns = isbns[isbns['rating'] > 99]
isbns.tail()

,isbn,rating
726,0786866586,100
727,0425136981,100
728,0312966806,100
729,0425189864,100
730,0425151867,100


In [8]:
print('Ratings before filtering:', len(df_ratings))

# remove users with less than 200 ratings
df_ratings = df_ratings[df_ratings['user'].isin(list(users['user']))]
print('Ratings after users filtering:', len(df_ratings))

# remove books with less than 100 ratings
df_ratings = df_ratings[df_ratings['isbn'].isin(list(isbns['isbn']))]
print('Ratings after books filtering:', len(df_ratings))

Ratings before filtering: 1149780
Ratings after users filtering: 527556
Ratings after books filtering: 49781


In [9]:
print('Unique books:', df_ratings.isbn.nunique())
print('Unique users:', df_ratings.user.nunique())

Unique books: 731
Unique users: 888


In [10]:
df_combined = pd.merge(df_ratings, df_books, on='isbn').reset_index()
df_combined = df_combined.drop(['index', 'author'], axis=1)
df_combined.tail()

,user,isbn,rating,title
49512,238864,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...
49513,251843,0399149325,1.0,Portrait of a Killer: Jack the Ripper -- Case ...
49514,253821,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...
49515,265115,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...
49516,266226,0399149325,0.0,Portrait of a Killer: Jack the Ripper -- Case ...


In [11]:
print('Unique books:', df_combined.isbn.nunique())
print('Unique users:', df_combined.user.nunique())

Unique books: 727
Unique users: 888


In [12]:
df_combined = df_combined.drop_duplicates(['user', 'isbn'])
df_combined_pivot = df_combined.pivot(index = 'isbn', columns = 'user', values = 'rating').fillna(0)
df_combined_matrix = csr_matrix(df_combined_pivot.values)
df_combined_pivot.tail()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
1573227331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1592400876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1878424319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

model.fit(df_combined_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
book_name = "The Queen of the Damned (Vampire Chronicles (Paperback))"
book_isbn = df_books[df_books['title'] == book_name].isbn.values[0]
book_data = df_combined_pivot.loc[book_isbn].to_numpy().reshape(1, -1)

distances, indices = model.kneighbors(book_data, n_neighbors=6)
print(distances)
print(indices)

[[5.9604645e-08 5.1784116e-01 5.3763384e-01 7.3450685e-01 7.4486566e-01
  7.9398352e-01]]
[[136 126 152 127 151 641]]


In [15]:
for index in indices[0]:
  book_isbn = df_combined_pivot.iloc[index,:].name
  print(df_books[df_books['isbn'] == book_isbn].title.values[0])

The Queen of the Damned (Vampire Chronicles (Paperback))
The Vampire Lestat (Vampire Chronicles, Book II)
The Tale of the Body Thief (Vampire Chronicles (Paperback))
Interview with the Vampire
The Witching Hour (Lives of the Mayfair Witches)
Catch 22


In [16]:
# function to return recommended books - this will be tested

def get_recommends(book = ""):
  book_isbn = df_books[df_books['title'] == book].isbn.values[0]
  book_data = df_combined_pivot.loc[book_isbn].to_numpy().reshape(1, -1)

  distances, indices = model.kneighbors(book_data, n_neighbors=6)
  recommended_books = [book, []]
  i = -1
  while i > -(distances.size):
     index = indices[0][i]
     book_isbn = df_combined_pivot.iloc[index,:].name
     book_name = df_books[df_books['isbn'] == book_isbn].title.values[0]
     recommended_book = [book_name, distances[0][i]]
     recommended_books[1].append(recommended_book)
     i -= 1

  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [17]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
